In [1]:
import pandas as pd
import numpy as np

import shap

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split

from ucimlrepo import fetch_ucirepo 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.utils.class_weight import compute_sample_weight

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
hepatitis = fetch_ucirepo(id=46) 

# data (as pandas dataframes) 
x = hepatitis.data.features 
y = hepatitis.data.targets -1

df = pd.concat([x,y],axis=1)#.dropna()

In [3]:
df_copy = df.copy()
missing_mask = df_copy.isna()
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_values = df_copy.copy()
imputed_values[:] = imputer.fit_transform(df_copy)
df_copy[missing_mask] = imputed_values[missing_mask]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_copy.loc[:, df_copy.columns != 'Class'], df_copy['Class'], test_size=0.3)

In [5]:

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train #provide your own target name
)


In [6]:
# Set class weights
#class_weights = {0: 1, 1: 10}  # Class 0: 1, Class 1: 10

# Set "scale_pos_weight" based on class balance
#pos_weight = sum(class_weights.values()) / sum(class_weights.keys())

# Train XGBoost model
#model = XGBClassifier(scale_pos_weight=pos_weight, class_weight=class_weights)


In [7]:
#Classification 

model = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1e-4, objective='binary:logistic')


#Model fit 
model.fit(X_train, y_train)

# Prediction
preds = model.predict(X_test)
preds


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [8]:
#Accuracy
np.mean((preds-y_test)**2)

0.1276595744680851

In [11]:
#Training error 
# Prediction
preds = model.predict(X_train)
np.mean((preds-y_train)**2)

0.24074074074074073

In [12]:
X100 = shap.utils.sample(X_train, len(X_train))


In [13]:
explainer = shap.Explainer(model.predict, X100)
shap_values = explainer(X_train)